In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
pd.options.display.max_columns = None
pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast
import warnings
import regex as re
warnings.filterwarnings('ignore')
from decimal import *
TWOPLACES = Decimal(10) ** -2
from fractions import Fraction
from natsort import natsorted

from enrich_dimensions.rounds import rounds, rounding, re_extract, curate, round_string_float, clean_data,reg_ex     
from enrich_dimensions.params import parameters, query_from_file
from enrich_dimensions.query_file import query_from_file 
from enrich_dimensions.custom import custom_field 

In [2]:
# # parameters
customer_id = '90'
formatted_attribute = 'Depth'
customer_name='%wbmason%'
buckets = """Office Chairs"""

strategy_version_input=762
attribute_id_input=2742

In [3]:
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [4]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)
    
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))    

def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

def find_first_digit(s, non=False):
    for i, x in enumerate(s):
        if x.isdigit() ^ non:
            return i
    return -1

def split_digits(s, case=False):
    non = True
    while s:
        i = find_first_digit(s, non)
        if i == 0:
            non = not non
        elif i == -1:
            yield int(s) if s.isdigit() else s if case else s.lower()
            s = ''
        else:
            x, s = s[:i], s[i:]
            yield int(x) if x.isdigit() else x if case else x.lower()

def natural_key(s, *args, **kwargs):
    return tuple(split_digits(s, *args, **kwargs))

def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

def atof(text):
    try:
        retval = float(text)
    except ValueError:
        retval = text
    return retval

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    float regex comes from https://stackoverflow.com/a/12643073/190597
    '''
    return [ atof(c) for c in re.split(r'[+-]?([0-9]+(?:[.][0-9]*)?|[.][0-9]+)', text) ]

def remove_exponent(num):
    return num.to_integral() if num == num.to_integral() else num.normalize()

def round_string_float(x):
    try:
        return remove_exponent(Decimal(x).quantize(TWOPLACES))
    except:
        return x  
    
def remove_frac(numbers):
    if str(numbers).find('/') > -1:  
        nom, den = numbers.split('/')
        tmp = float(nom)/float(den)
        return tmp
    else:
        return numbers
     
    
    
# Convert a fraction that is seperated by a space "1 1/3"
def remove_fraction(numbers):
    try:
        if numbers.find('/') > -1:   
            inte, fract = numbers.split(' ')
            nom, den = fract.split('/')
            tmp = float(nom)/float(den)
            num = int(inte) + tmp
            return num
        else:
            return numbers
    except:
        return numbers
    


#Convert a fraction that is seperated by a dash "1-1/2"
def remove_fractions(numbers):
    if numbers.find('-') > -1:
        numbers = literal_eval(numbers)
        inte, fract = numbers.split('-')
        nom, den = fract.split('/')
        tmp = float(nom)/float(den)
        num = int(inte) + tmp
        return str(num)
    else:
        return numbers
# tubes_new['height_new'] = tubes_new['height_new'].apply(lambda x: remove_fraction(x))

def frac2string(s):
    i, f = s.groups(0)
    f = round_string_float(Fraction(f))
    return str(int(i) + round_string_float(float(f)))

# measure='This is a fraction: 100 1/64'
# [re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, measure)]

In [5]:
import json
df = query_from_file(file_name='./query/custom_fields.sql', params=params)
# # df['custom_fields'] = df['custom_fields'].apply(lambda x: json.loads(x))
# custom_field_df=pd.json_normalize(df['custom_fields'])
# # custom_field_df = pd.json_normalize(df['custom_fields'])
# fields = ['Depth']
# length = pd.concat([df.drop('custom_fields', axis=1), custom_field_df[fields]], axis = 1)
# print(len(df))
length=df

In [6]:
del length['bucket_id']
del length['product_id']
del length['image_url']
del length['value']
del length['custom_fields']

In [7]:
dia=r'''(?i)(11.{0,3}x.?8.{0,5})|()'''
length['paper']=length['product_name'].apply(lambda x: re_extract(dia, str(x)))

# length['eigth'].explode().value_counts()
print(len(length[length['buckets'].astype(str)=='Binders & Accessories'][length['paper'].astype(str)!='[]']))
paper=length[length['buckets'].astype(str)=='Binders & Accessories'][length['paper'].astype(str)!='[]']
paper[curation_col]='8.5 in'
matchpaper=paper[['external_id',curation_col]]
# matchpaper

0


In [8]:
inches=r'''(?i)(8\.5.{0,3}x\s?11)|(8.?1\/2.{0,3}x\s?11)|()'''
length['eigth']=length['long_desc'].apply(lambda x: re_extract(inches, str(x)))
print(len(length[length['eigth'].astype(str)!='[]']))
eight=length[length['eigth'].astype(str)!='[]']
eight[curation_col]='8.5 in'
matcheight=eight[['external_id',curation_col]]
# matcheight

0


In [9]:
dia=r'''(?i)(?i)(8\.5.{0,3}x\s?11)|(8.?1\/2.{0,3}x\s?11\s?.{0,4})|()'''
length['eigth']=length['long_desc'].apply(lambda x: re_extract(dia, str(x)))
length['eigth'].explode().value_counts()
notebook=length[length['buckets'].astype(str)=='Notebooks, Journals, & Writing Pads'][length['eigth'].astype(str)!='[]']
notebook[curation_col]='8.5 in'
matchnote=notebook[['external_id',curation_col]]
# matchnote

In [10]:
labels=length[length['buckets'].astype(str)=='Labels, Stickers & Tickets']

dia=r'''(?i)(5.?1\/2"?\s?i?n?c?h?\shigh)|(5.?5"?\s?i?n?c?h?\shigh)|(\d+(?:\.\d+)?(.\d+\/\d+)\s?i?n?c?h?\shigh)|(\d.{0,3}diam)|(\d.{0,3}x\s?\d)|(\d.{0,3}expan)|()'''
length['eigth']=length['product_name'].apply(lambda x: re_extract(dia, str(x)))

dia=r'''(?i)(\d+\s?(?:\d+\/\d+|\.\d+).{0,2}x.?\d+\s?(?:\d+\/\d+|\.\d+)?.{0,2})|()'''
length['paper']=length['product_name'].apply(lambda x: re_extract(dia, str(x)))

length['eigth'].explode().value_counts()
print(len(length[length['buckets'].astype(str)=='Binders & Accessories'][length['paper'].astype(str)!='[]']))
binder=length[length['buckets'].astype(str)!='Binders & Accessories'][length['paper'].astype(str)!='[]']#['paper'].explode().value_counts()[0:500]
binder['paper']=binder['paper'].apply(lambda x: re.sub(r'''�''',' ',str(x))).apply(lambda x: re.sub(r'''\,.*|"|\[\'|w|\'\]|h''','',str(x)))
binder['paper']=binder['paper'].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, x)])
# binder['paper'].explode().value_counts()[0:500]

dia=r'''(?i)(8.5\s?x)|()'''
binder['split']=binder['paper'].apply(lambda x: re_extract(dia, str(x)))
eight=binder[binder['split'].astype(str)!='[]']
eight[curation_col]='8.5 in'
matcheight=eight[['external_id',curation_col]]
matcheight

0


Empty DataFrame
Columns: [external_id, Q:depth]
Index: []

In [11]:
# df_custom
# df_custom['match_cust']=df_custom['Depth [Nom]'].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, x)])
# df_custom['match_cust']=df_custom['match_cust'].apply(lambda x: re.sub(r'\.00','',str(x))).apply(lambda x: re.sub(r'\.10','.1',str(x))).apply(lambda x: re.sub(r'\.20','.2',str(x))).apply(lambda x: re.sub(r'\.30','.3',str(x))).apply(lambda x: re.sub(r'\.40','.4',str(x))).apply(lambda x: re.sub(r'\.50','.5',str(x))).apply(lambda x: re.sub(r'\.60','.6',str(x))).apply(lambda x: re.sub(r'\.70','.7',str(x))).apply(lambda x: re.sub(r'\.80','.8',str(x))).apply(lambda x: re.sub(r'\.90','.9',str(x)))
# df_custom['match_cust']=df_custom['match_cust'].apply(lambda x: re.sub(r'Adjustable|Up to |\d+.+� |�|\d.+- ','',str(x))).apply(lambda x: re.sub(r'"',' in',str(x)))
# df_custom['match_cust']=df_custom['match_cust'].apply(lambda x: re.sub(r'''\[\'|\'\]''','',str(x)))
# df_custom[curation_col]=df_custom['match_cust']
# regex_pattern_rounding=r'(?i)(\d+\.\d\d\d)|(\d+\.\d0)|(\,)|([a-eghj-mo-su-z])|(\.(?!\d))|(\d\d\d\d)|()'           
# df_custom['rounding'] = df_custom[curation_col].apply(lambda x: re_extract(regex_pattern_rounding, x))
# df_custom[df_custom['rounding'].astype(str)!='[]']

In [12]:
# matchcustom=df_custom[['external_id',curation_col]]
# print(len(pand))
# print(len(matchcustom))
# # matchcustom[curation_col].explode().value_counts()

In [13]:
import json
df = query_from_file(file_name='./query/custom_fields.sql', params=params)
# df['custom_fields'] = df['custom_fields'].apply(lambda x: json.loads(x))
custom_field_df=pd.json_normalize(df['custom_fields'])
# custom_field_df = pd.json_normalize(df['custom_fields'])
fields = ['Depth [Nom]']
length = pd.concat([df.drop('custom_fields', axis=1), custom_field_df[fields]], axis = 1)
print(len(df))

1193


In [14]:
print(len(length[(length['Depth [Nom]'].astype(str)=='[]')|(length['Depth [Nom]'].astype(str)=='nan')]))
df_cur=length[(length['Depth [Nom]'].astype(str)=='[]')|(length['Depth [Nom]'].astype(str)=='nan')]

1187


In [15]:
df_cur=length
regex_pattern_height=r'(?i)((?<!LabelManager)(?<!\-)(?<!\d)\d+\.\d+[\W]{0,3}de?p?t?h?(?!ia)\b)|(\d+.?\d+\/\d+[\W]{0,3}de?p?t?h?(?!ia)\b)|((?<!LabelManager.)(?<!\-)(?<!\d)(?<!\.)(?<!up.to.)(?<![\w])(?<!\/)(?<!\/)\d+(?!\.\d)(?<!\d\\\"\-\d\d)[\W]{0,3}de?p?t?h?(?!ia)\b)|()'        
df_cur['matches'] = df_cur['long_desc'].apply(lambda x: re_extract(regex_pattern_height, x))
print(len(df_cur[df_cur['matches'].astype(str)!='[]']))
depth=df_cur[df_cur['matches'].astype(str)!='[]']
depth['matches_depth']=depth['matches'].apply(lambda x: natsorted(x))

depth['matches_depth']=depth['matches_depth'].apply(lambda x: re.sub('131\/2\"d','13.5 in',str(x)))
depth['matches_depth']=depth['matches_depth'].apply(lambda x: re.sub('•|�','',str(x))).apply(lambda x: re.sub('''(?i)\"?\s?de?p?t?h?|\''\s?de?p?t?h?|"''',' in',str(x))).apply(lambda x: re.sub('"',' in',str(x))).apply(lambda x: re.sub('(?i)\s\s',' ',str(x)))
depth['matches_depth']=depth['matches_depth'].apply(lambda x: re.sub("\['",'',str(x))).apply(lambda x: re.sub("'\]",'',str(x)))#.apply(lambda x: re.sub("'\s?,\s?'",'',str(x)))
depth['matches_depth']=depth['matches_depth'].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, x)])
depth['matches_depth']=depth['matches_depth'].apply(lambda x: re.sub("\['",'["',str(x))).apply(lambda x: re.sub("'\]",'"]',str(x))).apply(lambda x: re.sub("'\s?,\s?'",'","',str(x)))
depth['matches_depth']=depth['matches_depth'].apply(lambda x: re.sub('in.?in','in',str(x))).apply(lambda x: re.sub("\'",' ft',str(x)))
depth['matches_depth']=depth['matches_depth'].apply(lambda x: re.sub('\.375','.38',str(x))).apply(lambda x: re.sub("\.125",'.13',str(x)))
depth['matches_depth']=depth['matches_depth'].apply(lambda x: re.sub(r'\.00','',str(x))).apply(lambda x: re.sub(r'\.10','.1',str(x))).apply(lambda x: re.sub(r'\.20','.2',str(x))).apply(lambda x: re.sub(r'\.30','.3',str(x))).apply(lambda x: re.sub(r'\.40','.4',str(x))).apply(lambda x: re.sub(r'\.50','.5',str(x))).apply(lambda x: re.sub(r'\.60','.6',str(x))).apply(lambda x: re.sub(r'\.70','.7',str(x))).apply(lambda x: re.sub(r'\.80','.8',str(x))).apply(lambda x: re.sub(r'\.90','.9',str(x)))  
depth['matches_depth']=depth['matches_depth'].apply(lambda x: re.sub('"5.75 in"\,"5.8 in"','"5.8"',str(x))).apply(lambda x: re.sub(r" \'",'',str(x))).apply(lambda x:re.sub(r'""','"',str(x)))




import ast
def remove_duplicates(A):
    [A.pop(count) for count,elem in enumerate(A) if A.count(elem)!=1]
    return A
depth['matches_depth']=depth['matches_depth'].apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(x)]))

depth['matches_depth']=depth['matches_depth'].apply(lambda x: re.sub(r"\[\'?",'["',str(x))).apply(lambda x: re.sub(r'\'?\]','"]',str(x))).apply(lambda x: re.sub(r'\'?\,\'?','","',str(x))).apply(lambda x: re.sub(r" \'",'',str(x)))
depth[curation_col]=depth['matches_depth']
rounding(depth, 'matches_depth','a-eghj-mo-su-z')

479


Empty DataFrame
Columns: [buckets, bucket_id, product_id, external_id, product_name, long_desc, url, image_url, value, Depth [Nom], matches, matches_depth, Q:depth, rounding]
Index: []

In [16]:
matchdepth=depth[['external_id',curation_col]]
# matchdepth

# Looking for new matches

In [17]:
# regex_pattern_length=r'(?i)(\d+\.\d+.?.?.?Le?n?g?t?h?\b)|(\d+.?\d+\/\d+.?.?.?Le?n?g?t?h?\b)|((?<!\.)(?<!\/)\d+(?!\.\d).?.?.?Le?n?g?t?h?\b)|((?<!\.)(?<!\/)\d+(?!\.).?.?.?Le?n?g?t?h?\b)|()'         
# df['matches_length'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_length, x))
regex_pattern_height=r'(?i)(\d+\.\d+[\W]?[\W]?[\W]?He?i?g?h?t?.?(?!Adjustable)\b)|(\d+.?\d+\/\d+[\W]?[\W]?[\W]?He?i?g?h?t?.?(?!Adjustable)\b)|((?<!\.)(?<!up.to.)(?<![\w])(?<!\/)(?<!\/)\d+(?!\.\d)(?<!\d\\?\"?\-\d\d)[\W]?[\W]?[\W]?He?i?g?h?t?.?(?!Adjustable)(?!.?setting)(?!.?position)(?!.?adjust)(?!.?-)(?!.?100%)\b)|()'        
df_cur['matches_height'] = df_cur['long_desc'].apply(lambda x: re_extract(regex_pattern_height, x))
# regex_pattern_width=r'(?i)(\d+\.\d+.?.?.?Wi?d?t?h?\b)|(\d+.?\d+\/\d+.?.?.?Wi?d?t?h?\b)|((?<!\.)(?<!\/)\d+(?!\.\d).?.?.?Wi?d?t?h?\b)|((?<!\.)(?<!\/)\d+(?!\.).?.?.?Wi?d?t?h?\b)|()'
# df['matches_width'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_width, x))
# regex_pattern_diameter=r'(?i)((?<!x )\d+\.\d+.?.?.?Diam?e?t?e?r?\b(?! x))|((?<!x )\d+.?\d+\/\d+.?.?.?Diam?e?t?e?r?\b(?! x))|((?<!x )(?<!\.)(?<!\/)\d+(?!\.\d).?.?.?Diam?e?t?e?r?\b(?! x))|((?<!x )(?<!\.)(?<!\/)\d+(?!\.).?.?.?Diam?e?t?e?r?\b(?! x))|()'
# df['matches_diameter'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_diameter, x))
# regex_pattern_ft=r"\d'.?.?.?[LWHD]|\d\'.?diameter|(\bround\b)|(circ)|(lamp)"
# df['matches_ft'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_ft, x))
# regex_pattern_blank=r'\d'
# df['new_matches'] = df['matches_diameter'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))
regex_pattern_blank=r'(?i)(\d+.?\d*\/?\d*.?I?n?c?h?"?.?x.?\d+.?\d*\/?\d*.?I?n?c?h?"?)|(\d+.?ft)|(\d+.?foot)|(\d+.?feet)|(\d+.?inch)|(\d+.?")|(\d+")|()'
df_cur['blank_matches'] = df_cur['product_name'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))
df_cur['blank_matches_long'] = df_cur['long_desc'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))

In [18]:
from natsort import natsorted
regex_pattern=r'''(?i)((?<!\-)(?<!\d)\d+(?:\.\d+|.\d+\/\d+)?[^\w\d]{0,3}He?i?g?h?t?(?!\sLabel)(?!.Adjustment)(?!.Adjustable)\b)|((?<!\.)(?<!up.to.)(?<![\w])(?<!\/)(?<!\/)(?<!\-)\d+(?!\.\d)(?<!\d\\\"\-\d\d)[\W]?[\W]?[\W]?He?i?g?h?t?(?!Adjustable)(?!\sLabel)(?!.?setting)(?!.?position)(?!.?adjust)(?!.?-)(?!.?100%)\b)|()'''           
df_cur['matches'] = df_cur['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
print(len(df_cur[df_cur['matches'].astype(str)!='[]']))

677


In [19]:
regex_pattern_blank=r'\d'
df_cur['new_matches'] = df_cur['matches'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))
height_new=df_cur[(df_cur['new_matches'].astype(str)!='[]')&(df_cur['matches'].astype(str)!='[]')]

height_new['matches']=height_new['matches'].apply(lambda x: natsorted(x))
height_new['matches']=height_new['matches'].apply(lambda x: re.sub(r'(?i)height',' in',str(x))).apply(lambda x: re.sub(r'(?i)height|\\" H|"H|H',' in',str(x))).apply(lambda x: re.sub(r'\\" H|"H|\"xa0H|xa0H|"|�|•','',str(x))).apply(lambda x: re.sub(r"\[|\]|'|\\",'',str(x))).apply(lambda x: re.sub(r"-",' ',str(x)))
height_new['matches']=height_new['matches'].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, x)])

height_new['matches']=height_new['matches'].apply(lambda x: re.sub('\\\" H',' in',str(x))).apply(lambda x: re.sub('\\\\','',str(x))).apply(lambda x: re.sub(',','","',str(x))).apply(lambda x: re.sub('" | "','"',str(x)))
import ast
def remove_duplicates(A):
    [A.pop(count) for count,elem in enumerate(A) if A.count(elem)!=1]
    return A
height_new['matches']=height_new['matches'].apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(x)]))
height_new['matches']=height_new['matches'].apply(lambda x: str(x)).apply(lambda x: re.sub("'",'"',str(x))).apply(lambda x: re.sub('" | "','"',str(x))).apply(lambda x: re.sub('  ',' ',str(x))).apply(lambda x: re.sub('\”|\)|inig','',str(x))).apply(lambda x: re.sub('  ',' ',str(x)))
del height_new['new_matches']

height_new['matches']=height_new['matches'].apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(str(x))]))
height_new['matches']=height_new['matches'].apply(lambda x: natsorted(ast.literal_eval(re.sub(r"'",'',str(re.sub(r'""','"',str(re.sub(r'" ','"',str(re.sub(r'  ',' ',str(re.sub(r',','","',str(re.sub(r'\]','"]',str(re.sub(r'\]','"]',str(re.sub(r'\[','["',str(x)))))))))))))))))))
height_new['matches']=height_new['matches'].apply(lambda x: re.sub(r"'",'',str(re.sub(r'""','"',str(re.sub(r'" ','"',str(re.sub(r'  ',' ',str(re.sub(r',','","',str(re.sub(r'\]','"]',str(re.sub(r'\]','"]',str(re.sub(r'\[','["',str(x)))))))))))))))))

In [20]:
height_new['matches']=height_new['matches'].apply(lambda x:re.sub(r'3.0 in','3 in',str(x)))
height_new['matches']=height_new['matches'].apply(lambda x: re.sub(r'"2.62 in","2.63 in"','"2.62 in"',str(x)))
height_new['matches']=height_new['matches'].apply(lambda x: re.sub(r'\.00','',str(x))).apply(lambda x: re.sub(r'\.10','.1',str(x))).apply(lambda x: re.sub(r'\.20','.2',str(x))).apply(lambda x: re.sub(r'\.30','.3',str(x))).apply(lambda x: re.sub(r'\.40','.4',str(x))).apply(lambda x: re.sub(r'\.50','.5',str(x))).apply(lambda x: re.sub(r'\.60','.6',str(x))).apply(lambda x: re.sub(r'\.70','.7',str(x))).apply(lambda x: re.sub(r'\.80','.8',str(x))).apply(lambda x: re.sub(r'\.90','.9',str(x)))  
print(len(height_new))
height_new[curation_col]=height_new['matches']
rounding(height_new, curation_col,'a-eghj-mo-su-z')
height_new=height_new[height_new['rounding'].astype(str)=='[]']
rounding(height_new, curation_col,'a-eghj-mo-su-z')

677


Empty DataFrame
Columns: [buckets, bucket_id, product_id, external_id, product_name, long_desc, url, image_url, value, Depth [Nom], matches, matches_height, blank_matches, blank_matches_long, Q:depth, rounding]
Index: []

In [21]:
matchesnew=height_new[['external_id',curation_col]]
matchesnew[curation_col].explode().value_counts()

["29 in"]                 106
["6 in"]                   47
["5 in"]                   38
["36 in"]                  36
["30 in"]                  24
["28 in"]                  22
["21 in"]                  20
["24 in"]                  18
["13.5 in"]                18
["37.25 in"]               15
["39.75 in"]               14
["20 in"]                  13
["15 in"]                  13
["16 in"]                  13
["42 in"]                  12
["18 in"]                  12
["65 in"]                  12
["49 in"]                  11
["37 in"]                  10
["29.5 in"]                10
["48 in"]                   9
["27.25 in"]                9
["16.5 in"]                 9
["29.62 in"]                9
["53 in"]                   8
["22.5 in"]                 8
["41 in"]                   8
["33.88 in"]                8
["31 in"]                   7
["38 in"]                   7
["12 in"]                   7
["38.06 in"]                6
["22 in"]                   6
["8.5 in"]

In [22]:
height_new[height_new['external_id'].astype(str)=='5637153108']

Empty DataFrame
Columns: [buckets, bucket_id, product_id, external_id, product_name, long_desc, url, image_url, value, Depth [Nom], matches, matches_height, blank_matches, blank_matches_long, Q:depth, rounding]
Index: []

# Compliance & Janitorial Signage

In [23]:
# df_sign=df[df['buckets'].astype(str)=='Compliance & Janitorial Signage']
# regex_pattern_blank=r'(?i)(x.?\d+.?\d\/\d")|(x.?\d+.?\d+")|(x.?\d+")|(x.?\d+.?\d\/\d'')|(x.?\d+.?\d+'')|(x.?\d+'')'
# df_sign['matches'] = df_sign['blank_matches'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))
# df_sign_new=df_sign[df_sign['matches'].astype(str)!='[]']


# df_sign_new['matches']=df_sign_new['matches'].apply(lambda x: natsorted(x))
# df_sign_new['matches']=df_sign_new['matches'].apply(lambda x: re.sub(r'''(?i)''|"''',' in',str(x))).apply(lambda x: re.sub(r'(?i)height|\\" H|"H|H',' in',str(x))).apply(lambda x: re.sub(r'(?i)x| |\"xa0H|xa0H|"|�|•','',str(x))).apply(lambda x: re.sub(r'in',' in',str(x))).apply(lambda x: re.sub(r"\[|\]|'|\\",'',str(x))).apply(lambda x: re.sub(r"-",' ',str(x)))
# df_sign_new['matches']=df_sign_new['matches'].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, x)])
# # df_new['matches']=df_new['matches'].apply(lambda x: round_string_float(x))
# df_sign_new['matches']=df_sign_new['matches'].apply(lambda x: re.sub('\\\" H',' in',str(x))).apply(lambda x: re.sub('\\\\','',str(x))).apply(lambda x: re.sub(',','","',str(x))).apply(lambda x: re.sub('" | "','"',str(x)))
# df_sign_new['matches']=df_sign_new['matches'].apply(lambda x: re.sub('''in']|']'''," in']",str(x)))
# import ast
# def remove_duplicates(A):
#     [A.pop(count) for count,elem in enumerate(A) if A.count(elem)!=1]
#     return A
# df_sign_new['matches']=df_sign_new['matches'].apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(x)]))
# df_sign_new['matches']=df_sign_new['matches'].apply(lambda x: str(x)).apply(lambda x: re.sub("'",'"',str(x))).apply(lambda x: re.sub('" | "','"',str(x))).apply(lambda x: re.sub('  ',' ',str(x))).apply(lambda x: re.sub('\”|\)|inig','',str(x))).apply(lambda x: re.sub('  ',' ',str(x)))

# df_sign_new['matches']=df_sign_new['matches'].apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(str(x))]))
# df_sign_new['matches']=df_sign_new['matches'].apply(lambda x: natsorted(ast.literal_eval(re.sub(r"'",'',str(re.sub(r'""','"',str(re.sub(r'" ','"',str(re.sub(r'  ',' ',str(re.sub(r',','","',str(re.sub(r'\]','"]',str(re.sub(r'\]','"]',str(re.sub(r'\[','["',str(x)))))))))))))))))))
# df_sign_new['matches']=df_sign_new['matches'].apply(lambda x: re.sub(r"'",'',str(re.sub(r'""','"',str(re.sub(r'" ','"',str(re.sub(r'  ',' ',str(re.sub(r',','","',str(re.sub(r'\]','"]',str(re.sub(r'\]','"]',str(re.sub(r'\[','["',str(x)))))))))))))))))
# df_sign_new

# Diameter

In [24]:
# df_diam=df[df['matches_diameter'].astype(str)!='[]']

# df_diam['matches']=df_diam['matches_diameter'].apply(lambda x: natsorted(x))
# df_diam['matches']=df_diam['matches'].apply(lambda x: re.sub(r'(?i)(".?diameter| diameter|".?dia|.?dia)',' in',str(x))).apply(lambda x: re.sub(r'(?i)diam|\\" H|"H|dia',' in',str(x))).apply(lambda x: re.sub(r'\\" H|"H|\"xa0H|xa0H|"|�|•','',str(x))).apply(lambda x: re.sub(r"\[|\]|'|\\",'',str(x))).apply(lambda x: re.sub(r"-",' ',str(x)))
# df_diam['matches']=df_diam['matches'].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, x)])
# # df_new['matches']=df_new['matches'].apply(lambda x: round_string_float(x))
# df_diam['matches']=df_diam['matches'].apply(lambda x: re.sub('\\\" H',' in',str(x))).apply(lambda x: re.sub('\\\\','',str(x))).apply(lambda x: re.sub(',','","',str(x))).apply(lambda x: re.sub('" | "','"',str(x)))
# import ast
# def remove_duplicates(A):
#     [A.pop(count) for count,elem in enumerate(A) if A.count(elem)!=1]
#     return A
# df_diam['matches']=df_diam['matches'].apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(x)]))
# df_diam['matches']=df_diam['matches'].apply(lambda x: str(x)).apply(lambda x: re.sub("'",'"',str(x))).apply(lambda x: re.sub('" | "','"',str(x))).apply(lambda x: re.sub('  ',' ',str(x))).apply(lambda x: re.sub('\”|\)|inig','',str(x))).apply(lambda x: re.sub('  ',' ',str(x)))
# # del height_new['new_matches']

# df_diam['matches']=df_diam['matches'].apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(str(x))]))
# df_diam['matches']=df_diam['matches'].apply(lambda x: natsorted(ast.literal_eval(re.sub(r"'",'',str(re.sub(r'""','"',str(re.sub(r'" ','"',str(re.sub(r'  ',' ',str(re.sub(r',','","',str(re.sub(r'\]','"]',str(re.sub(r'\]','"]',str(re.sub(r'\[','["',str(x)))))))))))))))))))
# df_diam['matches']=df_diam['matches'].apply(lambda x: re.sub(r"'",'',str(re.sub(r'""','"',str(re.sub(r'" ','"',str(re.sub(r'  ',' ',str(re.sub(r',','","',str(re.sub(r'\]','"]',str(re.sub(r'\]','"]',str(re.sub(r'\[','["',str(x)))))))))))))))))
# print(len(df_diam))
# # df_diam

In [25]:
# df_diam['matches']=df_diam['matches'].apply(lambda x: re.sub(r'''(?i)22.25","in''','22.25 in',str(x)))
# df_diam['matches']=df_diam['matches'].apply(lambda x: re.sub(r'''(?i)12\? in''','12 in',str(x)))
# df_diam['matches']=df_diam['matches'].apply(lambda x: re.sub(r'''(?i)8.312''','8.31',str(x)))

# df_diam[df_diam['matches'].astype(str)=='["12? in"]']

In [26]:
# df_diam[curation_col]=df_diam['matches']
# matches_dia=df_diam[['external_id',curation_col]]
# matches_dia

In [27]:
# print(len(df))
# print(len(df[(df['matches_diameter'].astype(str)=='[]')]))
# print(len(df[(df['blank_matches'].astype(str)=='[]')]))
# print(len(df[(df['blank_matches'].astype(str)=='[]')&(df['matches_diameter'].astype(str)=='[]')]))

In [28]:
# print(len(df[(df['blank_matches'].astype(str)=='[]')&(df['matches_diameter'].astype(str)=='[]')&(df['blank_matches_long'].astype(str)=='[]')]))

In [29]:
# df[(df['matches_diameter'].astype(str)=='[]')]

# NA Values

In [30]:
# regex_pattern_na=r'''(?i)((?<!Cord Measures.{10})\d+\.\d+.{0,2}['"].{0,2}[LWHD]?)|((?<!Cord Measures.{10})\d+.?\d+\/\d+.{0,2}['"].{0,2}[LWHD]?)|((?<!Cord Measures.{10})(?<!\.)(?<!\/)\d+(?!\.\d).{0,2}['"].{0,2}[LWHD]?)|((?<!Cord Measures.{10})(?<!\.)(?<!\/)\d+(?!\.).{0,2}['"].{0,2}[LWHD]?)|((?<!\.)\d+(?!\.\d).{0,3}?diam)|(measure)|()'''
# regex_pattern_measure=r'(?i)(measure)|()'
# df_na=df_cur
# df_na['na_matches'] = df_na['matches_depth'].apply(lambda x: re_extract(regex_pattern_na, x))
# df_na['na_measure_matches'] = df_na['long_desc'].apply(lambda x: re_extract(regex_pattern_measure, x))
# print(len(df_na[(df_na['na_matches'].astype(str)=='[]')&(df_na['matches_depth'].astype(str)=='[]')]))
df_na=df_cur[(df_cur['matches'].astype(str)=='[]')]
print(len(df_na))

516


In [31]:
df_na[curation_col]='n/a'
matchesna=df_na[['external_id',curation_col]]

In [32]:
# (\d+\.?\d*\\?\W?"?'?\W?f?t?I?n?c?h?\W?w)|(\d+\W?\d*\/\d*\\?\W?"?'?\W?f?t?I?n?c?h?\W?w)

# Min Max

In [33]:
# all_split=r'''(?i)(\d+\W?\d*\/?\d*\W?I?n?c?h?"?'?f?t?\W?[lwhd]?\b\W?x\W?\d+\W?\W?\d*\/?\W?\d*\W?I?n?c?h?"?'?f?t?\W?[lwhd]?\b\W?\W?x?\W?\d*\W?\d*\/?\d*\W?I?n?c?h?"?'?f?t?\W?[lwhd]?\b)|()'''
# pat_split_THREE=r'''(?i)(\W?\W?\d+.?\d*\/?\d*.?I?n?c?h?"?'?f?t?\W?[lwdh]?.?x.?\d+.?\d*\/?\d*.?I?n?c?h?"?'?f?t?\W?[lwdh]?.?x.?\d*.?\d+\/?\d*.?I?n?c?h?"?'?f?t?\W?[lwdh]?\w?)|()'''
# pat_split_TWO=r'''(?i)(\[\"\d+\.?\d*.?\w*.?"x\"\d+\.?\d*.?\w*.?"\])|()'''

# df['matches_split']=df['product_name'].apply(lambda x: re_extract(all_split, str(x)))
# df['matches_split_frac']=df['matches_split'].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, str(x))])
# df['matches_split_frac']=df['matches_split_frac'].apply(lambda x: re.sub(r'''�''','',str(x))).apply(lambda x: re.sub(r'''(\[\"\[\')|(\[\'\[\")|(\[\"\[\\\')''',"['[\'",str(x))).apply(lambda x: re.sub(r'''(\'\]\"\])|(\\\'\]\"\])|(\"\]\'\])''','''\']']''',str(x))).apply(lambda x: re.sub(r'''\"''',' in',str(x))).apply(lambda x: re.sub(r'''(\[\'\[\\\')|(\[\'\[\')''','["',str(x))).apply(lambda x: re.sub(r'''(\\\'\]\'\])|(\'\]\'\])''','"]',str(x))).apply(lambda x: re.sub(r'''(?i)x''','"x"',str(x))).apply(lambda x: re.sub(r'''\'''',' ft',str(x))).apply(lambda x: re.sub(r''' ''','',str(x))).apply(lambda x: re.sub(r'''\\ft\\ft''','ft',str(x))).apply(lambda x: re.sub(r'''\\ft''','ft',str(x))).apply(lambda x: re.sub(r'''X''','x',str(x))).apply(lambda x: re.sub(r'''in''',' in',str(x))).apply(lambda x: re.sub(r'''ft''',' ft',str(x))).apply(lambda x: re.sub(r'''\,.*''','"]',str(x)))                                                                                                                                                             

# df['matches_split_three'] = df['matches_split_frac'].apply(lambda x: re_extract(pat_split_THREE, str(x)))
# df['matches_split_two'] = df['matches_split_frac'].apply(lambda x: re_extract(pat_split_TWO, str(x)))


# regex_pattern_blank=r'(?i)(\d+\.\d*.?w\b)|()'
# df['matches'] = df['matches_split_frac'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))
# split=df[df['matches_split'].astype(str)!='[]']
# print(len(split))
# print(len(split[split['matches_split_two'].astype(str)!='[]']))
# print(len(split[split['matches_split_three'].astype(str)!='[]']))

In [34]:
# pat_call_cout=r'''(?i)(\d+\.?\d*.?.?l\b)|()'''
# split['matches']=split['matches_split_frac'].apply(lambda x: re_extract(pat_call_cout, str(x)))

# print(len(split[split['matches'].astype(str)!='[]']))
# width_call_out=split[split['matches'].astype(str)!='[]']
# width_call_out['matches']=width_call_out['matches'].apply(lambda x: re.sub(r'''(?i)h''',' in',str(x))).apply(lambda x: re.sub(r'''\[''','["',str(x))).apply(lambda x: re.sub(r'''\]''','"]',str(x))).apply(lambda x: re.sub(r'''(')''','',str(x)))
# width_call_out[curation_col]=width_call_out['matches']
# matcheswidthcallout=width_call_out[['external_id',curation_col]]

In [35]:
# # split['matches_split_two'].explode().value_counts()[0:500]
# print(len(split[split['matches_split_two'].astype(str)!='[]']))
# split_two=split[(split['matches_split_two'].astype(str)!='[]')&(split['matches'].astype(str)=='[]')]
# print(len(split_two))

In [36]:
# pat_split_width_inch=r'''(?i)(ft)|()'''
# split_two['ft']=split_two['matches_split_two'].apply(lambda x: re_extract(pat_split_width_inch, str(x)))
# split_two_inch=split_two[split_two['ft'].astype(str)=='[]']
# print(len(split_two_inch))
# split_two_inch.head(2)

In [37]:
# split['matches_split_frac'].explode().value_counts()[0:500]

In [38]:
# pat_split_length=r'''(?i)(x\"\d+\.?\d*.{0,4}"\])|()'''
# pat_split_width=r'''(?i)(\[\"\d+\.?\d*.{0,4}x)|()'''
# split_two_inch['length']=split_two_inch['matches_split_two'].apply(lambda x: re_extract(pat_split_length, str(x)))
# split_two_inch['width']=split_two_inch['matches_split_two'].apply(lambda x: re_extract(pat_split_width, str(x)))

In [39]:
# split_two_inch['length']=split_two_inch['length'].apply(lambda x: re.sub(r'''\[|\]|x|\"|in|'| ''','',str(x)))
# split_two_inch['width']=split_two_inch['width'].apply(lambda x: re.sub(r'''\[|\]|x|\"|in|'| ''','',str(x)))
# # split_two_inch

In [40]:
# split_two_inch['length']=split_two_inch['length'].apply(lambda x: re.sub(r'''12.5di''','12.5',str(x))).apply(lambda x: re.sub(r'''10\.5\?''','10.5',str(x))).apply(lambda x: re.sub(r'''0\.62\-''','0.62',str(x))).apply(lambda x: re.sub(r'''18\-''','18',str(x)))
# split_two_inch['length']=split_two_inch['length'].apply(lambda x: re.sub(r'\?|\-| ','',str(x)))
# split_two_inch['width']=split_two_inch['width'].apply(lambda x: re.sub(r'\?|\-','',str(x)))

In [41]:
# pat_call_cout=r'''(?i)(\d+\.?\d*.?.?[a-z]\b)|()'''
# split_two_inch['matches']=split_two_inch['length'].apply(lambda x: re_extract(pat_call_cout, str(x)))
# split_two_inch=split_two_inch[split_two_inch['matches'].astype(str)=='[]']

In [42]:
# print(len(split_two_inch))
# split_two_inch=split_two_inch[split_two_inch['buckets'].astype(str)!='Compliance & Janitorial Signage']
# print(len(split_two_inch))
# print(split_two_inch['buckets'].explode().value_counts())

In [43]:
# split_two_inch['length']=split_two_inch['length'].astype(float)
# split_two_inch['width']=split_two_inch['width'].astype(float)

# split_two_inch['Max'] = split_two_inch[['length', 'width']].values.max(1)
# split_two_inch['Min'] = split_two_inch[['length', 'width']].values.min(1)
# split_two_inch['Max']=split_two_inch['Max'].astype(float).apply(lambda x: remove_exponent(Decimal(x).quantize(TWOPLACES)))

In [44]:
# split_two_inch=split_two_inch[split_two_inch['length'].astype(str)!=""]
# split_two_inch=split_two_inch[split_two_inch['width'].astype(str)!=""]
# split_two_inch['length']=split_two_inch['length'].apply(lambda x:re.sub(r' in','',str(x)))#.apply(lambda x:str(f'"{x}"'))
# split_two_inch['width']=split_two_inch['width'].apply(lambda x:re.sub(r' in','',str(x))).apply(lambda x: re.sub(r'','',str(x)))
# split_two_inch['length']=split_two_inch['length'].astype(float)
# split_two_inch['width']=split_two_inch['width'].astype(float)

# split_two_inch['Max'] = split_two_inch[['length', 'width']].values.max(1)
# split_two_inch['Min'] = split_two_inch[['length', 'width']].values.min(1)
# split_two_inch['Max']=split_two_inch['Max'].astype(float).apply(lambda x: remove_exponent(Decimal(x).quantize(TWOPLACES)))
# split_two_inch['Min']=split_two_inch['Min'].astype(float).apply(lambda x: remove_exponent(Decimal(x).quantize(TWOPLACES)))

In [45]:
# split_two_inch[curation_col]=split_two_inch['Max']
# msplit=split_two_inch[['external_id',curation_col]]
# msplit[curation_col]=split_two_inch[curation_col].apply(lambda x: str(f"['{x} in']")).apply(lambda x: re.sub(r"'",'"',str(x)))
# # msplit[curation_col].explode().value_counts()

In [46]:
# print(len(df))
# print(len(matchesna)+len(matchesnew)+len(matchcustom))

In [47]:
# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][1]
#     return name
# print(get_df_name(matcheight))

# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][0]
#     return name
# print(get_df_name(matchcustom))
# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][0]
#     return name
# print(get_df_name(matchesna))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchdepth))

# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][1]
#     return name
# print(get_df_name(matcheight))


matchdepth


In [48]:
stop 

NameError: name 'stop' is not defined

# send to the folder for upload

In [51]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBYmatchesna - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/{customer} - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 

In [52]:
looks_good('WBMason', attribute, df, matchdepth)

In [55]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBYmatchesna - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/{customer} - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 

looks_good('WBMason', attribute, df, matchesna)

In [57]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBYmatchesna - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/{customer} - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
looks_good('WBMason', attribute, df, matchdepth)

In [58]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBYmatchesna - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/{customer} - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
looks_good('WBMason', attribute, df, matcheight)

In [60]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBYmatchesna - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/{customer} - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
looks_good('WBMason', attribute, df, matchnote)

In [61]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBYmatchesna - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/{customer} - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
looks_good('WBMason', attribute, df, matchpaper)

In [62]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][1]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBYmatchesna - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/{customer} - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
looks_good('WBMason', attribute, df, matcheight)